<a href="https://colab.research.google.com/github/jcmeve/Travoard/blob/main/MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.display import Math
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import json
import IPython
from google.colab import output
import requests
import folium
from folium.plugins import MiniMap

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/Colab Notebooks/total_score.csv'
data = pd.read_csv(filename)
data = data.sample(frac=1).reset_index(drop=True)
data.head()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,documents__category_group_code,documents__category_group_name,documents__category_name,documents__distance,time,tag,score(0~10)
0,MT1,대형마트,"가정,생활 > 대형마트",3405,점심,관광,0
1,AT4,관광명소,"여행 > 관광,명소 > 테마거리",1181,오후,관광,10
2,FD6,음식점,"음식점 > 한식 > 해물,생선 > 굴,전복",4877,오후,맛집,3
3,MT1,대형마트,"가정,생활 > 대형마트",3078,점심,관광,1
4,MT1,대형마트,"가정,생활 > 대형마트",3532,저녁,관광,3


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop('score(0~10)', axis=1)
y = data['score(0~10)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print("X_train's shape : ", X_train.shape)
print("y_train's shape : ", y_train.shape)
print("\nX_test's shape : ", X_test.shape)
print("y_test's shape : ", y_test.shape)




X_train's shape :  (4374, 6)
y_train's shape :  (4374,)

X_test's shape :  (486, 6)
y_test's shape :  (486,)


In [4]:

class ContextDataSet(Dataset):
  def __init__(self, target_type, time, tag, before_distance,score):
    ###one hot encoding

    type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
    self.target_type = []
    for i in range(len(target_type)):
      self.target_type.insert(i,type_converter[target_type[i]])


    time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
    self.time = []
    for i in range(len(time)):
      self.time.insert(i, time_converter[time[i]])

    tag_converter = {'쇼핑':0,'관광':1,'맛집':2}
    self.tag = []
    for i in range(len(tag)):
      self.tag.insert(i, tag_converter[tag[i]])


    self.before_distance = before_distance
    self.score = score




    self.target_type = nn.functional.one_hot(torch.tensor(self.target_type),6)
    self.time = nn.functional.one_hot(torch.tensor(self.time),6)
    self.tag = nn.functional.one_hot(torch.tensor(self.tag),3)




  def __len__(self):
    return len(self.score)
  def __getitem__(self,idx):
    feature = []

    for i in range(len(self.target_type[idx])):
      feature.append(self.target_type[idx][i])
    for i in range(len(self.time[idx])):
      feature.append(self.time[idx][i])
    for i in range(len(self.tag[idx])):
      feature.append(self.tag[idx][i])
    feature.append(self.before_distance[idx]*0.0001)


    return feature, self.score[idx]

In [5]:

train_set = ContextDataSet(
    X_train['documents__category_group_code'].values.tolist(),
    X_train['time'].values.tolist(),
    X_train['tag'].values.tolist(),
    X_train['documents__distance'].values.tolist(),
    y_train.to_list()
    )
train_loader = DataLoader(train_set, batch_size = 1, shuffle = True)

test_set = ContextDataSet(
    X_test['documents__category_group_code'].values.tolist(),
    X_test['time'].values.tolist(),
    X_test['tag'].values.tolist(),
    X_test['documents__distance'].values.tolist(),
    y_test.to_list()
    )
test_loader = DataLoader(test_set, batch_size = 1, shuffle = True)

In [6]:
feature,lable = next(iter(train_loader))
print(feature, lable)

[tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([0.3346], dtype=torch.float64)] tensor([0])


In [7]:
### 날씨는 ai안쓰고 거름

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=16, out_features=128, bias = False),
            nn.Sigmoid(),
        )

        self.layer2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=32, bias = False),
        )

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=32, out_features=4, bias = False),
        )


        self.layer4 = nn.Sequential(
            nn.Linear(in_features=4, out_features=1, bias = False),
        )


    def forward(self, x):
        x = x.to(device)
        logits = self.layer1(x)
        logits = self.layer2(logits)
        logits = self.layer3(logits)
        logits = self.layer4(logits)
        return logits

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [10]:
model = NeuralNetwork().to(device) # device로 Network 전송
print(model) # 모델 구조 확인

NeuralNetwork(
  (layer1): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=False)
    (1): Sigmoid()
  )
  (layer2): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=32, out_features=4, bias=False)
  )
  (layer4): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=False)
  )
)


In [11]:
# hyperparameter 설정
import torch.optim as optim

#criterion = nn.CrossEntropyLoss(|) # loss function
criterion = nn.MSELoss() # loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # optimizer

n_epoch = 10  # the number of epochs
n_batch = 10 # the number of batches

In [12]:
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        inputs, labels = data

        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)

        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizer의 파라미터 gradient를 0으로 설정
        optimizer.zero_grad()

        pred = model(inputs).to(device)#예측값 forward

        loss = criterion(pred, labels) #loss값

        loss.backward() #backward
        optimizer.step() #optimize
        #############################################

        # loss 출력
        running_loss += loss.item()

        if i % n_batch == 0:    # print every n_batch mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / n_batch:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.404
[1,    11] loss: 20.885
[1,    21] loss: 1.231
[1,    31] loss: 13.581
[1,    41] loss: 10.975
[1,    51] loss: 8.074
[1,    61] loss: 13.589
[1,    71] loss: 25.727
[1,    81] loss: 22.616
[1,    91] loss: 6.371
[1,   101] loss: 9.094
[1,   111] loss: 12.614
[1,   121] loss: 38.053
[1,   131] loss: 10.601
[1,   141] loss: 19.418


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,   151] loss: 14.502
[1,   161] loss: 17.636
[1,   171] loss: 23.026
[1,   181] loss: 15.048
[1,   191] loss: 17.958
[1,   201] loss: 11.151
[1,   211] loss: 16.353
[1,   221] loss: 10.361
[1,   231] loss: 10.903
[1,   241] loss: 24.207
[1,   251] loss: 21.936
[1,   261] loss: 28.687
[1,   271] loss: 9.489
[1,   281] loss: 15.373
[1,   291] loss: 24.492
[1,   301] loss: 13.181
[1,   311] loss: 19.792
[1,   321] loss: 4.330
[1,   331] loss: 10.488
[1,   341] loss: 7.171
[1,   351] loss: 14.188
[1,   361] loss: 10.653
[1,   371] loss: 4.586
[1,   381] loss: 11.566
[1,   391] loss: 7.594
[1,   401] loss: 11.057
[1,   411] loss: 15.144
[1,   421] loss: 21.639
[1,   431] loss: 12.089
[1,   441] loss: 14.175
[1,   451] loss: 13.452
[1,   461] loss: 20.806
[1,   471] loss: 4.484
[1,   481] loss: 4.381
[1,   491] loss: 9.694
[1,   501] loss: 24.693
[1,   511] loss: 11.157
[1,   521] loss: 19.426
[1,   531] loss: 6.747
[1,   541] loss: 6.107
[1,   551] loss: 18.965
[1,   561] loss: 14.722
[1

In [13]:
# 전체 test data에 대한 결과 확인
correct = 0
total = 0

diff_sum = 0

with torch.no_grad(): # 모델을 학습하는 것이 아니므로 gradient 계산을 할 필요가 없음
    for data in test_loader:
        inputs, labels = data

        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        #_, predicted = torch.max(outputs.data, 1)

        predicted = outputs
        print(predicted.item())
        print(labels.item())
        diff = predicted.item() - labels.item()
        diff_sum+= diff

        total += labels.size(0)


        correct += (predicted == labels.to(device)).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
diff_avg = diff_sum / total
print(diff_avg)

2.8846435546875
2.0
2.175133228302002
3.0
0.36580324172973633
0.0
4.378105163574219
6.0
2.6871395111083984
1.0
0.2395901083946228
1.0
3.7359633445739746
4.0
1.8792846202850342
1.0
6.305533409118652
10.0
3.9768314361572266
4.0
0.7666366696357727
0.0
0.8718898296356201
0.0
3.5804555416107178
1.0
4.200354099273682
2.0
1.2726314067840576
2.0
4.7785115242004395
8.0
2.8363516330718994
2.0
5.015791416168213
3.0
2.5922293663024902
1.0
5.206280708312988
8.0
3.4557929039001465
4.0
4.278426170349121
9.0
5.616788864135742
2.0
4.044918060302734
5.0
2.7646327018737793
6.0
2.0130691528320312
1.0
1.841532826423645
3.0
5.179432392120361
4.0
7.239923477172852
8.0
3.227829933166504
6.0
5.935537338256836
4.0
-0.4860735535621643
0.0
3.1032066345214844
4.0
4.8228864669799805
10.0
6.1499505043029785
8.0
2.500412702560425
3.0
-0.7845585346221924
0.0
6.246207237243652
5.0
-0.8769470453262329
0.0
3.642103433609009
3.0
-0.7905505299568176
0.0
1.2721459865570068
1.0
-0.13199453055858612
0.0
0.8161675930023193
0.0

In [14]:
def place_finder(category,y,x):
    url = 'https://dapi.kakao.com/v2/local/search/category.json'
    params = {'category_group_code': category,'radius' : 10000, 'y' :y, 'x' : x}
    headers = {"Authorization": "KakaoAK 9a12f840d08a0b698f040ffc93f18211"}

    places = requests.get(url, params=params, headers=headers).json()['documents']

    return places

In [28]:
from folium.plugins import MousePosition


m = folium.Map(location=[37.284,127.043], zoom_start=17)
formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
MousePosition(    position="topright",
    separator=" | ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,).add_to(m)
display(m)

In [29]:
print("x좌표를 입력하세요")
x = input()

print("y좌표를 입력하세요")
y = input()


place_data = []
categories = ['AT4','MT1','CT1','AD5','FD6','CE7']
for cate in categories:
  places = place_finder(cate, y,x)
  for i in range(len(places)):
    place_data.append(places[i])

print('아침,오전,점심,오후,저녁,밤 중 하나를 고르세요')
time_raw = input()
print('쇼핑,관광,맛집 중 하나를 고르세요')
tag_raw = input()

x좌표를 입력하세요
127.043
y좌표를 입력하세요
37.284
아침,오전,점심,오후,저녁,밤 중 하나를 고르세요
아침
쇼핑,관광,맛집 중 하나를 고르세요
맛집


In [30]:
print(len(place_data))

90


In [31]:
with torch.no_grad():
    for i in range(len(place_data)):

        type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
        group_code_raw = place_data[i].get("category_group_code")
        group_code = type_converter[group_code_raw]



        time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
        time = time_converter[time_raw]


        tag_converter = {'쇼핑':0,'관광':1,'맛집':2}
        tag = tag_converter[tag_raw]



        distance = place_data[i].get("distance")



        group_code = nn.functional.one_hot(torch.tensor(group_code),6)
        time = nn.functional.one_hot(torch.tensor(time),6)
        tag = nn.functional.one_hot(torch.tensor(tag),3)

        feature = []

        for j in range(len(group_code)):
          feature.append(group_code[j])
        for j in range(len(time)):
          feature.append(time[j])
        for j in range(len(tag)):
          feature.append(tag[j])
        feature.append(float(distance)*0.0001)


        inputs = feature
        inputs = torch.tensor([inputs], dtype = torch.float32)
        inputs = inputs.to(device)

        predicted = model(inputs)

        place_data[i]["score"] = predicted.item()
        group_code_name = place_data[i].get("category_group_name")

ret = sorted(place_data, key=lambda place_data: place_data["score"], reverse=True)
for i in range(10):
  print(ret[i])

{'address_name': '경기 수원시 영통구 원천동 18-1', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 패스트푸드 > 맥도날드', 'distance': '571', 'id': '25001083', 'phone': '070-7209-0922', 'place_name': '맥도날드 수원아주대점', 'place_url': 'http://place.map.kakao.com/25001083', 'road_address_name': '경기 수원시 영통구 아주로 46', 'x': '127.04398690068008', 'y': '37.27891570780273', 'score': 9.566993713378906}
{'address_name': '경기 수원시 팔달구 우만동 54', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 일식 > 돈까스,우동', 'distance': '575', 'id': '223879734', 'phone': '031-213-6822', 'place_name': '한조카츠', 'place_url': 'http://place.map.kakao.com/223879734', 'road_address_name': '경기 수원시 팔달구 중부대로239번길 78', 'x': '127.0421410248533', 'y': '37.27886682211022', 'score': 9.56135082244873}
{'address_name': '경기 수원시 팔달구 우만동 185', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기', 'distance': '596', 'id': '12811062', 'phone': '031-212-92

In [32]:
m = folium.Map(location=[float(ret[0]['y']),float(ret[0]['x'])], zoom_start=17)

for i in range(10):
    folium.Marker([ret[i]['y'],ret[i]['x']],
              tooltip=ret[i]['place_name'],
              ).add_to(m)
display(m)
